举例1：大模型分析工具的调用

In [ ]:
# 1、获取大模型
# 导入相关依赖
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


# 定义LLM模型
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2、获取工具的列表
tools = [MoveFileTool()]

# 3、因为大模型invoke调用时，需要传入函数的列表，所以需要将工具转换为函数
functions = [convert_to_openai_function(t) for t in tools]

# 4、获取消息列表
messages = [HumanMessage(content="将文件a移动到桌面")]

# 5、调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools, #不支持
    functions = functions,
)
print(response)

作为对比:

In [ ]:
# 获取消息列表
messages = [HumanMessage(content="查询一下明天北京的天气")]

# 调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools, #不支持
    functions = functions,
)

print(response)

通过上面两个测试发现，得到的AIMessage的核心属性如下：

1、如果分析出需要调用对应的工具：

content: 信息为空。因为大模型要调用工具，所以就不会直接返回信息给用户

additional_kwargs: 包含 function_call 字段，指明具体函数调用的参数和函数名。比如：

additional_kwargs={'function_call': {'arguments': '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}', 'name': 'move_file'}, 'refusal': None}

2. 如果分析出不需要调用对应的工具：

content: 信息不为空。

additional_kwargs: 不包含

# 举例2：如何调用具体大模型分析出来的工具

说明：

1、大模型与Agent的核心区别：是否涉及到工具的调用

2、针对于大模型：仅能分析出要调用的工具，但是此工具（或函数）不能真正的执行

针对于Agent: 除了分析出要调用的工具之外，还可以执行具体的工具（或函数

In [ ]:
# 1、获取大模型
# 导入相关依赖
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


# 定义LLM模型
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2、获取工具的列表
tools = [MoveFileTool()]

# 3、因为大模型invoke调用时，需要传入函数的列表，所以需要将工具转换为函数
functions = [convert_to_openai_function(t) for t in tools]

# 4、获取消息列表
messages = [HumanMessage(content="将文件a移动到桌面")]

# 5、调用大模型（传入消息列表、工具的列表）
response = chat_model.invoke(
    input=messages,
    # tools = tools, #不支持
    functions = functions,
)
print(response)

# 分析下要调用哪个工具或函数

In [ ]:
if "function_call" in response.additional_kwargs:
    tool_name = response.additional_kwargs["function_call"]["name"]
    tool_args = response.additional_kwargs["function_call"]["arguments"]
    print(f"调用工具: {tool_name} \n参数: {tool_args}")

else :

    print(f"模型回复: {response.content}")